In [4]:
!pip install pandas


In [5]:
!pip install openai

In [6]:
import pandas as pd
import openai
import time
import logging
from typing import Optional
#from error_handler import robust_summarize, APIErrorHandler
#from usage_monitor import usage_monitor

In [16]:
logger = logging.getLogger(__name__)

class ArticleSummarizer:
    """LLM API를 사용하여 CSV 파일에 요약을 추가하는 클래스"""
    
    def __init__(self, api_key: str = None, model: str = "gpt-4o"):
        if api_key is None:
            api_key = os.getenv("OPENAI_API_KEY")
        self.client = openai.OpenAI(api_key=api_key)
        self.model = model
        
    def _expand_short_text(self, text: str) -> str:
        prompt = f"""
다음은 뉴스 입니다. 이를 바탕으로 100자 내외로 요약한 내용을 생성해서 한글로 주세요:

"{text}"

"""
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "당신은 뉴스를 요약하는 전문가입니다."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300,
            temperature=0.5
        )
        
        return response.choices[0].message.content.strip()

    def summarize_text(self, text: str) -> str:
        """
        텍스트 요약 (직접 OpenAI API 호출)
        """
        prompt = f"""
다음은 뉴스 입니다. 이를 바탕으로 100자 내외로 요약한 내용을 생성해서 한글로 주세요:

"{text}"
"""
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "당신은 뉴스를 요약하는 전문가입니다."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300,
            temperature=0.5
        )
        return response.choices[0].message.content.strip()

    
    def process_csv(self, 
                   csv_path: str,
                   text_column: str = "content",
                   output_path: Optional[str] = None) -> str:
        """
        CSV 파일의 텍스트를 요약하여 새 컬럼 추가
        
        Args:
            csv_path: 입력 CSV 파일 경로
            text_column: 요약할 텍스트 컬럼명
            output_path: 출력 파일 경로 (None이면 자동 생성)
            
        Returns:
            출력 파일 경로
        """
        # CSV 파일 읽기
        df = pd.read_csv(csv_path)
        print(f"CSV 파일 로드: {len(df)}개 행")
        
        # 출력 경로 설정
        if output_path is None:
            output_path = csv_path.replace('.csv', '_with_summaries.csv')
        
        # 요약 컬럼 추가
        df['summary'] = ''
        
        # 각 행에 대해 요약 생성
        for i in range(len(df)):
            text = df.iloc[i][text_column]
            
            print(f"요약 진행중: {i+1}/{len(df)}")
            
            try:
                summary = self.summarize_text(text)
                df.iloc[i, df.columns.get_loc('summary')] = summary
            except Exception as e:
                print(f"행 {i} 요약 실패: {e}")
                df.iloc[i, df.columns.get_loc('summary')] = "요약 실패"
            
            # API 호출 제한을 위한 지연
            time.sleep(1)
        
        # 결과 저장
        df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
        print(f"요약 완료! 결과 파일: {output_path}")
        
        return output_path

In [17]:
import os
import dotenv
dotenv.load_dotenv()

# ArticleSummarizer 인스턴스 생성
summarizer = ArticleSummarizer(model="gpt-4o")  # api_key는 None이면 .env에서 자동 로드

# CSV 파일 경로 지정
csv_path = "test_articles.csv"  # 또는 "answer/test_articles.csv" 경로에 따라 수정

# 요약 실행
output_path = summarizer.process_csv(
    csv_path=csv_path,
    text_column="content",  # 기사 내용 컬럼명
    output_path=None        # None이면 자동으로 _with_summaries.csv 생성
)

# 결과 확인
import pandas as pd
result_df = pd.read_csv(output_path)
print(result_df.head())

CSV 파일 로드: 5개 행
요약 진행중: 1/5
요약 진행중: 2/5
요약 진행중: 3/5
요약 진행중: 4/5
요약 진행중: 5/5
요약 완료! 결과 파일: test_articles_with_summaries.csv
   index              title  \
0      1   AI 기술 발전과 일자리 변화   
1      2  기후변화 대응 신재생에너지 확산   
2      3      반도체 업계 공급망 재편   
3      4     MZ세대 소비 트렌드 변화   
4      5  도심 항공 모빌리티 상용화 임박   

                                             content reporter        date  \
0  인공지능 기술의 급속한 발전으로 전 세계 노동시장에 큰 변화가 일어나고 있다. 맥킨...     기술일보  2025-08-26   
1  전 세계적으로 기후변화 대응을 위한 신재생에너지 투자가 급증하고 있다. 국제에너지기...     환경일보  2025-08-26   
2  글로벌 반도체 업계에 공급망 재편 바람이 거세게 불고 있다. 미국과 중국 간의 기술...     산업일보  2025-08-26   
3  MZ세대(밀레니얼+Z세대)의 소비 패턴이 전통적인 소비 문화를 크게 바꾸고 있다. ...    소비자일보  2025-08-26   
4  하늘을 나는 택시로 불리는 도심 항공 모빌리티(UAM) 서비스가 현실화되고 있다. ...     교통일보  2025-08-26   

                                             summary  
0  인공지능 발전으로 2030년까지 3억 7500만 개의 일자리가 사라지지만, 기술 분...  
1  전 세계 신재생에너지 투자가 급증하며 2024년 1조 8000억 달러를 넘어섰다. ...  
2  미중 기술 경쟁 속 각국이 반도체 공급망 자국 회귀를 추진 중이다. 미국, 유럽, ...  
3  MZ세대